In [1]:
# app.py
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
import streamlit as st

# Function to create sequences
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

# Streamlit app
st.title("📈 AI Stock Trend Predictor")

ticker = st.text_input("Enter Stock Symbol (e.g., AAPL, MSFT, TSLA)", "AAPL")
start_date = st.date_input("Start Date", pd.to_datetime("2015-01-01"))
end_date = st.date_input("End Date", pd.to_datetime("2023-01-01"))

if st.button("Run Prediction"):
    df = yf.download(ticker, start=start_date, end=end_date)

    # Add indicators
    df['MA50'] = df['Close'].rolling(50).mean()
    df['RSI'] = 100 - (100 / (1 + df['Close'].pct_change().rolling(14).mean()))
    df = df.dropna()
    
    # Preprocess
    features = df[['Close', 'MA50', 'RSI']].values
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(features)

    X, y = create_dataset(scaled)
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Build LSTM
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

    # Predict
    preds = model.predict(X_test)
    pred_price = scaler.inverse_transform(np.hstack((preds, np.zeros((len(preds), 2)))))[:, 0]
    actual_price = scaler.inverse_transform(np.hstack((y_test.reshape(-1,1), np.zeros((len(y_test), 2)))))[:, 0]

    # Generate trade signals
    signals = ['Buy' if p > a else 'Sell' if p < a else 'Hold' for a, p in zip(actual_price, pred_price)]

    # Plot
    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(actual_price, label="Actual")
    ax.plot(pred_price, label="Predicted")
    colors = ['green' if s == 'Buy' else 'red' if s == 'Sell' else 'gray' for s in signals]
    ax.scatter(range(len(signals)), pred_price, c=colors, alpha=0.6, label="Signal")
    ax.set_title(f"{ticker} Prediction and Trade Signals")
    ax.set_xlabel("Time")
    ax.set_ylabel("Price")
    ax.legend()
    st.pyplot(fig)

    # Show signals
    st.subheader("📊 Recent Trade Suggestions:")
    signal_df = pd.DataFrame({
        'Day': list(range(-10, 0)),
        'Actual Price': actual_price[-10:],
        'Predicted Price': pred_price[-10:],
        'Signal': signals[-10:]
    })
    st.dataframe(signal_df.set_index("Day"))


2025-06-21 20:09:56.665 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-21 20:09:59.723 
  command:

    streamlit run C:\Users\Dirck\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-21 20:09:59.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-21 20:09:59.728 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-21 20:09:59.728 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-21 20:09:59.729 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-21 20:09:59.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-21 20:09: